In [185]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsapi
import json
import re
import datetime

from tqdm import tqdm

%matplotlib inline

In [267]:
def generate_teams_dict():
    team_params = {'activeStatus':'Y', 'season':2019, 'sportIds':1, 'fields':'teams,id,name,teamCode,fileCode,teamName,locationName,shortName,venue'}
    teams_info = statsapi.get('teams', team_params)

    teams_dict = {}
    for team in teams_info['teams']:
        teams_dict[team['fileCode']] = team['id']
    return teams_dict

In [268]:
def get_yesterdays_games():
    yesterday = datetime.datetime.today() - datetime.timedelta(days=1)
    return yesterday.strftime("%m/%d/%Y")

In [307]:
def get_todays_games():
    return statsapi.schedule(datetime.datetime.today().strftime("%m/%d/%Y"))

def matchups_and_prob_pitchers_dicts(day='today'):
    games_today = get_todays_games()
    if day == 'yesterday':
        games_today = get_yesterdays_games()
        
    home_away_dict = {}
    prob_pitchers_dict = {}
    for game in games_today:
        prob_pitchers_dict[game['away_id']] = game['away_probable_pitcher']
        prob_pitchers_dict[game['home_id']] = game['home_probable_pitcher']
        
        home_away_dict[game['home_id']] = game['away_id']
        
    away_home_dict = {v:k for k, v in home_away_dict.items()}
    return home_away_dict, away_home_dict, prob_pitchers_dict

In [312]:
def get_player_list(team_id):
    player_names = []
    roster = statsapi.roster(team_id)
    roster_list = roster.split("\n")[:-1]
    for player in roster_list:
        player_names.append(" ".join(player.split()[2:]))
    return player_names

get_player_list(137)

['Alex Dickerson',
 'Austin Slater',
 'Brandon Belt',
 'Brandon Crawford',
 'Buster Posey',
 'Donovan Solano',
 'Drew Pomeranz',
 'Jeff Samardzija',
 'Joe Panik',
 'Kevin Pillar',
 'Madison Bumgarner',
 'Mark Melancon',
 'Mike Yastrzemski',
 'Pablo Sandoval',
 'Reyes Moronta',
 'Sam Coonrod',
 'Sam Dyson',
 'Shaun Anderson',
 'Stephen Vogt',
 'Tony Watson',
 'Trevor Gott',
 'Tyler Austin',
 'Tyler Beede',
 'Will Smith',
 'Zach Green']

In [309]:
def get_player_id_from_name(player_name):
    try:
        return statsapi.lookup_player(player_name)[0]['id']
    except IndexError:
        return False

def check_pos_player(player_name):
    try:
        return statsapi.lookup_player(player_name)[0]['primaryPosition']['abbreviation'] != "P"
    except IndexError:
        return False

def get_current_season_stats(player_name):

    if not check_pos_player(player_name):
        raise ValueError("Player name entered is not a position player")
        
    player_id = get_player_id_from_name(player_name)
    stats_dict = {"Name": player_name, "ID": player_id, 
                  "Team": statsapi.lookup_player(player_id)[0]['currentTeam']['id']}
    
    # Look up the player's current season hitting stats
    get_player_stats = statsapi.player_stats(player_id, 'hitting') 
    
    # Get the stats for the most recent season
    curr_season_stats = get_player_stats.split("Season Hitting")[-1]
    
    #Break up the stats into a list
    stats_list = curr_season_stats.split("\n")[1:-2]
    for stat in stats_list:
        stat_name = re.search("[A-Za-z]+", stat).group()
        stat_val = re.search("[^:A-Za-z]+", stat).group()
        stats_dict[stat_name] = float(stat_val)
    return stats_dict

In [323]:
# These functions were defined with the help of toddrob99 on github, who developed the
# MLB-StatsAPI module. I made a post on reddit.com/r/mlbdata, which he mantains to 
# answer questions about making API calls for specific purposes. I asked how to get stats
# over the past x days and how to get head-to-head batting stats. The post is linked
# here: https://www.reddit.com/r/mlbdata/comments/cewwfo/getting_headtohead_batting_stats_and_last_x_games/?

def batting_past_N_days(N, player_id, end_date=datetime.datetime.today()):
    
    start_date = (end_date - datetime.timedelta(days=N)).strftime("%m/%d/%Y")
    end_date = end_date.strftime("%m/%d/%Y")
    hydrate = ('stats(group=[hitting],type=[byDateRange],startDate={},endDate={}),currentTeam'.
               format(start_date, end_date))
    
    params = {'personId': player_id, 'hydrate':hydrate}
    
    try: 
        r = statsapi.get('person',params)
        batting_stats = r['people'][0]['stats'][0]['splits'][0]['stat']
    except ValueError: 
        
    
    # Only get rate stats for past N days
    filtered = {k + "_p{}d".format(N):float(v) for k, v in batting_stats.items() 
                if type(v) == str 
                and k != 'stolenBasePercentage'
                or k == 'hits'} 
    
    return filtered

def get_h2h_vs_pitcher(batter_id, opponent_id):
    
    hydrate = 'stats(group=[hitting],type=[vsPlayer],opposingPlayerId={},season=2019,sportId=1)'.format(opponent_id)
    params = {'personId': batter_id, 'hydrate':hydrate, 'sportId':1}
    r = statsapi.get('person',params)
    try: 
        batting_stats = r['people'][0]['stats'][1]['splits'][0]['stat']
    except KeyError:
        return {'atBats_h2h': 0.0, 'avg_h2h': 0.0, 'hits_h2h': 0.0,
               'obp_h2h': 0.0, 'ops_h2h': 0.0, 'slg_h2h': 0.0}
    
    # Only get rate stats vs pitcher
    filtered = {k + "_h2h":float(v) for k, v in batting_stats.items() 
                if type(v) == str 
                and k != 'stolenBasePercentage'
                or k == 'hits'
                or k == 'atBats'} 
    
    return filtered

In [324]:
get_h2h_vs_pitcher(get_player_id_from_name("Buster Posey"), get_player_id_from_name("Blake Snell"))

{'atBats_h2h': 0.0,
 'avg_h2h': 0.0,
 'hits_h2h': 0.0,
 'obp_h2h': 0.0,
 'ops_h2h': 0.0,
 'slg_h2h': 0.0}

In [320]:
hydrate = 'stats(group=[hitting],type=[vsPlayer],opposingPlayerId={},season=2019,sportId=1)'.format(get_player_id_from_name("Blake Snell"))
params = {'personId': get_player_id_from_name("Buster Posey"), 'hydrate':hydrate, 'sportId':1}
r = statsapi.get('person',params)
r

{'copyright': 'Copyright 2019 MLB Advanced Media, L.P.  Use of any content on this page acknowledges agreement to the terms posted here http://gdx.mlb.com/components/copyright.txt',
 'people': [{'active': True,
   'batSide': {'code': 'R', 'description': 'Right'},
   'birthCity': 'Leesburg',
   'birthCountry': 'USA',
   'birthDate': '1987-03-27',
   'birthStateProvince': 'GA',
   'boxscoreName': 'Posey',
   'currentAge': 32,
   'draftYear': 2008,
   'firstLastName': 'Buster Posey',
   'firstName': 'Gerald',
   'fullFMLName': 'Gerald Dempsey Posey',
   'fullLFMName': 'Posey, Gerald Dempsey',
   'fullName': 'Buster Posey',
   'gender': 'M',
   'height': "6' 1",
   'id': 457763,
   'initLastName': 'B Posey',
   'isPlayer': True,
   'isVerified': True,
   'lastFirstName': 'Posey, Buster',
   'lastInitName': 'Posey, B',
   'lastName': 'Posey',
   'link': '/api/v1/people/457763',
   'middleName': 'Dempsey',
   'mlbDebutDate': '2009-09-11',
   'nameFirstLast': 'Buster Posey',
   'nameSlug': 'b

In [328]:
def pitching_past_N_days(N, player_id, end_date=datetime.datetime.today()):
    
    start_date = (end_date - datetime.timedelta(days=N)).strftime("%m/%d/%Y")
    end_date = end_date.strftime("%m/%d/%Y")
    hydrate = 'stats(group=[pitching],type=[byDateRange],startDate={},endDate={}),currentTeam'.format(start_date, end_date)
    
    params = {'personId': player_id, 'hydrate':hydrate}
    r = statsapi.get('person',params)
    pitching_stats = r['people'][0]['stats'][0]['splits'][0]['stat']
    
    # Only get rate stats for past N days
    filtered = {k + "_p{}d".format(N):float(v) for k, v in pitching_stats.items() 
                if type(v) == str and v != ".---"} 
    
    return filtered

In [289]:
def check_pitcher_right_handed(pitcher_id):
    try:
        params = {'personId': pitcher_id}
        r = statsapi.get('person',params)
        return r['people'][0]['pitchHand']['code'] == 'R'
    except IndexError:
        return False

In [290]:
def check_batter_right_handed(batter_id):
    try:
        params = {'personId': batter_id}
        r = statsapi.get('person',params)
        return r['people'][0]['batSide']['code'] == 'R'
    except IndexError:
        return False

In [291]:
def check_pitcher_batter_opposite_hand(batter_id, pitcher_id):
    return check_pitcher_right_handed(pitcher_id) != check_batter_right_handed(batter_id)

In [365]:
def player_got_hit_in_game(player_id, game_id, home_or_away):
    
    params = {'gamePk':game_id,
      'fields': 'gameData,teams,teamName,shortName,teamStats,batting,atBats,runs,hits,rbi,strikeOuts,baseOnBalls,leftOnBase,players,boxscoreName,liveData,boxscore,teams,players,id,fullName,batting,avg,ops,era,battingOrder,info,title,fieldList,note,label,value'}
    r = statsapi.get('game', params)
    player_stats = r['liveData']['boxscore']['teams'][home_or_away]['players'].get('ID' + str(player_id), False)
    if not player_stats: 
        return False 
    else:
        return player_stats['stats']['batting'].get('hits', 0) > 0

In [379]:
def convert_to_FL_format(name):
    last_first = name.split(",")
    last_first.reverse()
    last_first[0] = last_first[0].strip()
    return " ".join(last_first)

In [380]:
convert_to_FL_format("Perez, Martin")

'Martin Perez'

In [360]:
params = {'gamePk':566715,
  'fields': 'gameData,teams,teamName,shortName,teamStats,batting,atBats,runs,hits,rbi,strikeOuts,baseOnBalls,leftOnBase,players,boxscoreName,liveData,boxscore,teams,players,id,fullName,batting,avg,ops,era,battingOrder,info,title,fieldList,note,label,value'}
r = statsapi.get('game', params)
luke = get_player_id_from_name("Luke Maile")
r['liveData']['boxscore']['teams']['away']['players']

{'ID456034': {'gameStatus': {},
  'jerseyNumber': '10',
  'parentTeamId': 111,
  'person': {'fullName': 'David Price', 'id': 456034},
  'position': {},
  'seasonStats': {'batting': {'atBats': 2,
    'avg': '.500',
    'baseOnBalls': 0,
    'hits': 1,
    'leftOnBase': 1,
    'ops': '1.000',
    'rbi': 1,
    'runs': 0,
    'strikeOuts': 1}},
  'stats': {'batting': {}},
  'status': {}},
 'ID488768': {'gameStatus': {},
  'jerseyNumber': '48',
  'parentTeamId': 111,
  'person': {'fullName': 'Andrew Cashner', 'id': 488768},
  'position': {},
  'seasonStats': {'batting': {'atBats': 3,
    'avg': '.000',
    'baseOnBalls': 0,
    'hits': 0,
    'leftOnBase': 2,
    'ops': '.000',
    'rbi': 0,
    'runs': 1,
    'strikeOuts': 0}},
  'stats': {'batting': {}},
  'status': {}},
 'ID502110': {'allPositions': [{}],
  'battingOrder': '400',
  'gameStatus': {},
  'jerseyNumber': '28',
  'parentTeamId': 111,
  'person': {'fullName': 'J.D. Martinez', 'id': 502110},
  'position': {},
  'seasonStats': 

In [293]:
def get_opposing_pitcher(player_id, game_id):
    teams = statsapi.get('schedule', {'sportId': '1', 'gamePk': game_id, 
                                      'hydrate':'probablePitcher'})['dates'][0]['games'][0]['teams']
    home_team_id = teams['home']['team']['id']
    away_team_id = teams['away']['team']['id']
    
    home_prob_pitcher = teams['home']['probablePitcher']['fullName']
    away_prob_pitcher = teams['away']['probablePitcher']['fullName']
    
    if statsapi.lookup_player(player_id)[0]['currentTeam']['id'] == home_team_id:
        return away_prob_pitcher
    else: 
        return home_prob_pitcher

In [330]:
def batting_past_N_games(N, player_id):
    hydrate = 'stats(group=[hitting],type=[lastXGames],limit={}),currentTeam'.format(N)
    
    params = {'personId': player_id, 'hydrate':hydrate}
    r = statsapi.get('person',params)
    batting_stats = r['people'][0]['stats'][0]['splits'][0]['stat']
    
    # Only get rate stats for past N days
    filtered = {k + "_p{}G".format(N):float(v) for k, v in batting_stats.items() 
                if type(v) == str 
                and k != 'stolenBasePercentage'
                or k == 'hits'} 
    
    return filtered

def pitching_past_N_games(N, player_id):
    hydrate = 'stats(group=[pitching],type=[lastXGames],limit={}),currentTeam'.format(N)
    
    params = {'personId': player_id, 'hydrate':hydrate}
    r = statsapi.get('person',params)
    pitching_stats = r['people'][0]['stats'][0]['splits'][0]['stat']
    
    # Only get rate stats for past N days
    filtered = {k + "_p{}G".format(N):float(v) for k, v in pitching_stats.items() 
                if type(v) == str and v != ".---"} 
    
    return filtered

In [336]:
player_got_hit_in_game(get_player_id_from_name("Aaron Judge"), 567512)

True

In [313]:
pitching_past_N_games(5, get_player_id_from_name("Jacob DeGrom"))

{'avg_p5G': 0.232,
 'era_p5G': 2.32,
 'groundOutsToAirouts_p5G': 0.91,
 'hitsPer9Inn_p5G': 7.55,
 'homeRunsPer9_p5G': 0.58,
 'inningsPitched_p5G': 31.0,
 'pitchesPerInning_p5G': 16.39,
 'runsScoredPer9_p5G': 2.32,
 'stolenBasePercentage_p5G': 0.6,
 'strikePercentage_p5G': 0.67,
 'strikeoutWalkRatio_p5G': 3.82,
 'strikeoutsPer9Inn_p5G': 12.19,
 'walksPer9Inn_p5G': 3.19,
 'whip_p5G': 1.19,
 'winPercentage_p5G': 0.5}

In [205]:
statsapi.lookup_player("Trevor Gott")

[{'boxscoreName': 'Gott',
  'currentTeam': {'id': 137},
  'firstLastName': 'Trevor Gott',
  'firstName': 'Trevor',
  'fullFMLName': 'Trevor Vaughan Gott',
  'fullLFMName': 'Gott, Trevor Vaughan',
  'fullName': 'Trevor Gott',
  'id': 641627,
  'initLastName': 'T Gott',
  'lastFirstName': 'Gott, Trevor',
  'lastInitName': 'Gott, T',
  'lastName': 'Gott',
  'mlbDebutDate': '2015-06-14',
  'nameFirstLast': 'Trevor Gott',
  'primaryNumber': '58',
  'primaryPosition': {'abbreviation': 'P', 'code': '1'},
  'useName': 'Trevor'}]

In [316]:
get_current_season_stats("Kevin Pillar").keys()

dict_keys(['Name', 'ID', 'Team', 'gamesPlayed', 'groundOuts', 'runs', 'doubles', 'triples', 'homeRuns', 'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch', 'avg', 'atBats', 'obp', 'slg', 'ops', 'caughtStealing', 'stolenBases', 'stolenBasePercentage', 'groundIntoDoublePlay', 'numberOfPitches', 'plateAppearances', 'totalBases', 'rbi', 'leftOnBase', 'sacBunts', 'sacFlies', 'babip', 'groundOutsToAirouts'])

In [325]:
get_h2h_vs_pitcher(get_player_id_from_name("Buster Posey"), get_player_id_from_name("Blake Snell"))

{'atBats_h2h': 0.0,
 'avg_h2h': 0.0,
 'hits_h2h': 0.0,
 'obp_h2h': 0.0,
 'ops_h2h': 0.0,
 'slg_h2h': 0.0}

In [233]:
(statsapi.schedule("07/21/2019")[10]['game_id'])

566520

In [311]:
generate_teams_dict()

{'ana': 108,
 'ari': 109,
 'atl': 144,
 'bal': 110,
 'bos': 111,
 'chc': 112,
 'cin': 113,
 'cle': 114,
 'col': 115,
 'cws': 145,
 'det': 116,
 'hou': 117,
 'kc': 118,
 'la': 119,
 'mia': 146,
 'mil': 158,
 'min': 142,
 'nym': 121,
 'nyy': 147,
 'oak': 133,
 'phi': 143,
 'pit': 134,
 'sd': 135,
 'sea': 136,
 'sf': 137,
 'stl': 138,
 'tb': 139,
 'tex': 140,
 'tor': 141,
 'was': 120}

In [300]:
batting_past_N_games(30, get_player_id_from_name("Brandon Crawford"))

{'avg_p30d': 0.284,
 'hits_p30d': 31.0,
 'obp_p30d': 0.336,
 'ops_p30d': 0.841,
 'slg_p30d': 0.505}

In [340]:
statsapi.schedule(yesterday)

[{'away_id': 115,
  'away_name': 'Colorado Rockies',
  'away_pitcher_note': '',
  'away_probable_pitcher': 'Lambert, Peter',
  'away_score': '0',
  'current_inning': '',
  'doubleheader': 'N',
  'game_date': '2019-07-22',
  'game_datetime': '2019-07-22T23:05:00Z',
  'game_id': 565942,
  'game_num': 1,
  'game_type': 'R',
  'home_id': 120,
  'home_name': 'Washington Nationals',
  'home_pitcher_note': 'Fedde returned to the Nationals rotation with a solid start in Baltimore last week, tossing six innings of one-run ball with a pair of strikeouts, although he was pulled after just 66 pitches. He is 0-2 with a 12.38 ERA in two career starts against Colorado.',
  'home_probable_pitcher': 'Fedde, Erick',
  'home_score': '0',
  'inning_state': '',
  'status': 'Postponed',
  'summary': '2019-07-22 - Colorado Rockies @ Washington Nationals (Postponed)'},
 {'away_id': 138,
  'away_name': 'St. Louis Cardinals',
  'away_pitcher_note': 'Ponce de Leon will get another shot at the Pirates -- and hope

In [367]:
pitching_past_N_games(5, get_player_id_from_name('Perez, Martin'))

ValueError: Request failed. Status Code: 400.

In [371]:
get_player_id_from_name("Perez, Martin")

False

In [383]:
# This cell generates rows for a DataFrame of batting stats--one row per player

yesterday = (datetime.datetime.today() - datetime.timedelta(days = 1)).strftime("%m/%d/%Y")
today = datetime.datetime.today().strftime("%m/%d/%Y")

rows_list = []
for game in tqdm(statsapi.schedule(yesterday)):
    
    if game['status'] not in ['In Progress', 'Final']:
        continue
    
    game_id = game['game_id']
    away_id = game['away_id']
    home_id = game['home_id']
    home_player_list = get_player_list(home_id)
    away_player_list = get_player_list(away_id)
    
    away_prob_Pname = convert_to_FL_format(game['away_probable_pitcher'])
    home_prob_Pname = convert_to_FL_format(game['home_probable_pitcher'])
    
    away_probable_pitcher = get_player_id_from_name(away_prob_Pname)
    home_probable_pitcher = get_player_id_from_name(home_prob_Pname)
    
    away_pitcher_p5G = pitching_past_N_games(5, away_probable_pitcher)
    home_pitcher_p5G = pitching_past_N_games(5, home_probable_pitcher)
    
    for player in home_player_list:
        print(player)
        player_id = get_player_id_from_name(player)
        try:
            new_row = list(get_current_season_stats(player).values())
            new_row += list(batting_past_N_games(7, player_id).values())
            new_row += list(batting_past_N_games(15, player_id).values())
            new_row += list(away_pitcher_p5G.values())
            new_row += list(get_h2h_vs_pitcher(player_id, away_probable_pitcher).values())
            new_row.append(float(check_pitcher_batter_opposite_hand(batter_id=player_id, 
                                                                  pitcher_id=away_probable_pitcher)))
            new_row.append(float(player_got_hit_in_game(game_id=game_id, player_id=player_id, home_or_away='home')))
            rows_list.append(new_row)
        except (ValueError, IndexError):
            continue

    for player in away_player_list:
        print(player)
        player_id = get_player_id_from_name(player)
        try:
            new_row = list(get_current_season_stats(player).values())
            new_row += list(batting_past_N_games(7, player_id).values())
            new_row += list(batting_past_N_games(15, player_id).values())
            new_row += list(home_pitcher_p5G.values())
            new_row += list(get_h2h_vs_pitcher(player_id, home_probable_pitcher).values())
            new_row.append(float(check_pitcher_batter_opposite_hand(batter_id=player_id, 
                                                                  pitcher_id=home_probable_pitcher)))
            new_row.append(float(player_got_hit_in_game(game_id=game_id, player_id=player_id, home_or_away='away')))
            rows_list.append(new_row)
        except (ValueError, IndexError):
            continue















  0%|          | 0/15 [00:00<?, ?it/s]

Adam Eaton
Andrew Stevenson
Anibal Sanchez
Anthony Rendon
Brian Dozier
Erick Fedde
Fernando Rodney
Gerardo Parra
Howie Kendrick
Javy Guerra
Joe Ross
Juan Soto
Kurt Suzuki
Matt Adams
Matt Grace
Michael Blazek
Patrick Corbin
Sean Doolittle
Stephen Strasburg
Tanner Rainey
Tony Sipp
Trea Turner
Victor Robles
Wander Suero
Yan Gomes
Bryan Shaw
Carlos Estevez
Chad Bettis
Charlie Blackmon
Chris Iannetta
Daniel Murphy
David Dahl
Garrett Hampson
German Marquez
Ian Desmond
Jairo Diaz
Jake McGee
Jeff Hoffman
Jon Gray
Kyle Freeland
Nolan Arenado
Peter Lambert
Raimel Tapia
Ryan McMahon
Sam Howard
Scott Oberg
Tony Wolters
Trevor Story
Wade Davis
Yonder Alonso















  7%|▋         | 1/15 [01:02<14:29, 62.13s/it]

Adam Frazier
Bryan Reynolds
Chris Archer
Chris Stratton
Clay Holmes
Colin Moran
Corey Dickerson
Dario Agrazal
Elias Diaz
Felipe Vazquez
Francisco Liriano
Jacob Stallings
Joe Musgrove
Jordan Lyles
Jose Osuna
Josh Bell
Jung Ho Kang
Kevin Newman
Kyle Crick
Luis Escobar
Melky Cabrera
Michael Feliz
Richard Rodriguez
Starling Marte
Trevor Williams
Adam Wainwright
Andrew Knizner
Andrew Miller
Carlos Martinez
Dakota Hudson
Daniel Ponce de Leon
Dexter Fowler
Giovanny Gallegos
Harrison Bader
Jack Flaherty
John Brebbia
John Gant
Jose Martinez
Kolten Wong
Matt Wieters
Michael Wacha
Mike Mayers
Miles Mikolas
Paul DeJong
Paul Goldschmidt
Rangel Ravelo
Tommy Edman
Tyler O'Neill
Tyler Webb
Yairo Munoz















 13%|█▎        | 2/15 [02:06<13:36, 62.82s/it]

...














 87%|████████▋ | 13/15 [13:44<02:09, 64.73s/it]

Anthony Bass
Austin Nola
Cory Gearrin
Daniel Vogelbach
Domingo Santana
Dylan Moore
Erik Swanson
J.P. Crawford
Kristopher Negron
Kyle Seager
Mallex Smith
Marco Gonzales
Matt Carasiti
Matt Magill
Matt Wisler
Mike Leake
Omar Narvaez
Roenis Elias
Sam Tuivailala
Tim Beckham
Tim Lopes
Tom Murphy
Tommy Milone
Wade LeBlanc
Yusei Kikuchi
Adrian Sampson
Ariel Jurado
Asdrubal Cabrera
Brett Martin
Chris Martin
Danny Santana
Delino DeShields
Elvis Andrus
Hunter Pence
Jeff Mathis
Jesse Chavez
Joey Gallo
Jose Leclerc
Kyle Bird
Lance Lynn
Logan Forsythe
Mike Minor
Nomar Mazara
Pedro Payano
Phillips Valdez
Rafael Montero
Rougned Odor
Shin-Soo Choo
Taylor Guerrieri
Tim Federowicz















 93%|█████████▎| 14/15 [14:42<01:02, 62.84s/it]

A.J. Pollock
Alex Verdugo
Austin Barnes
Caleb Ferguson
Casey Sadler
Clayton Kershaw
Cody Bellinger
Corey Seager
David Freese
Enrique Hernandez
Hyun-Jin Ryu
Joc Pederson
Joe Kelly
JT Chargois
Julio Urias
Justin Turner
Kenley Jansen
Kenta Maeda
Matt Beaty
Max Muncy
Pedro Baez
Ross Stripling
Russell Martin
Walker Buehler
Yimi Garcia
Adalberto Mejia
Albert Pujols
Andrelton Simmons
Brian Goodwin
Cam Bedrosian
David Fletcher
Dustin Garneau
Felix Pena
Griffin Canning
Hansel Robles
Jaime Barria
Justin Anderson
Justin Upton
Kevan Smith
Kole Calhoun
Luis Garcia
Luis Rengifo
Matt Thaiss
Mike Trout
Noe Ramirez
Shohei Ohtani
Taylor Cole
Taylor Ward
Trevor Cahill
Ty Buttrey















100%|██████████| 15/15 [15:53<00:00, 65.13s/it]

In [387]:
get_h2h_vs_pitcher(get_player_id_from_name("Kevin Pillar"), get_player_id_from_name("Jacob DeGrom"))

{'atBats_h2h': 3.0,
 'avg_h2h': 0.333,
 'hits_h2h': 1.0,
 'obp_h2h': 0.333,
 'ops_h2h': 1.0,
 'slg_h2h': 0.667}

In [398]:
player_stats_columns

['Name',
 'ID',
 'Team',
 'gamesPlayed',
 'groundOuts',
 'runs',
 'doubles',
 'triples',
 'homeRuns',
 'strikeOuts',
 'baseOnBalls',
 'intentionalWalks',
 'hits',
 'hitByPitch',
 'avg',
 'atBats',
 'obp',
 'slg',
 'ops',
 'caughtStealing',
 'stolenBases',
 'stolenBasePercentage',
 'groundIntoDoublePlay',
 'numberOfPitches',
 'plateAppearances',
 'totalBases',
 'rbi',
 'leftOnBase',
 'sacBunts',
 'sacFlies',
 'babip',
 'groundOutsToAirouts',
 'hits_p7G',
 'avg_p7G',
 'obp_p7G',
 'slg_p7G',
 'ops_p7G',
 'hits_p15G',
 'avg_p15G',
 'obp_p15G',
 'slg_p15G',
 'ops_p15G',
 'avg_p5G',
 'stolenBasePercentage_p5G',
 'era_p5G',
 'inningsPitched_p5G',
 'whip_p5G',
 'strikePercentage_p5G',
 'groundOutsToAirouts_p5G',
 'winPercentage_p5G',
 'pitchesPerInning_p5G',
 'strikeoutWalkRatio_p5G',
 'strikeoutsPer9Inn_p5G',
 'walksPer9Inn_p5G',
 'hitsPer9Inn_p5G',
 'runsScoredPer9_p5G',
 'homeRunsPer9_p5G',
 'hits_h2h',
 'avg_h2h',
 'atBats_h2h',
 'obp_h2h',
 'slg_h2h',
 'ops_h2h',
 'pitcher_hitter_opposite

In [447]:
sample_hitter = get_player_id_from_name("Kevin Pillar")
sample_pitcher = get_player_id_from_name("Jacob DeGrom")
player_stats_columns = list(get_current_season_stats("Kevin Pillar").keys())
player_stats_columns += list(batting_past_N_games(7, sample_hitter).keys())
player_stats_columns += list(batting_past_N_games(15, sample_hitter).keys())
player_stats_columns += list(pitching_past_N_games(5, sample_pitcher).keys())
player_stats_columns += list(get_h2h_vs_pitcher(sample_hitter, sample_pitcher).keys())
player_stats_columns += ['pitcher_hitter_opposite_hand', 'player_got_hit']

In [449]:
player_stats_table = pd.DataFrame(data=rows_list, columns=player_stats_columns)
player_stats_table.head()

,Name,ID,Team,gamesPlayed,groundOuts,runs,doubles,triples,homeRuns,strikeOuts,...,runsScoredPer9_p5G,homeRunsPer9_p5G,hits_h2h,avg_h2h,atBats_h2h,obp_h2h,slg_h2h,ops_h2h,pitcher_hitter_opposite_hand,player_got_hit
0,Adam Eaton,594809,120,95.0,89.0,57.0,14.0,4.0,6.0,66.0,...,5.88,2.77,1.0,0.500,2.0,0.667,1.000,1.667,1.0,1.0
1,Anthony Rendon,543685,120,85.0,66.0,71.0,28.0,2.0,20.0,52.0,...,5.88,2.77,0.0,0.000,3.0,0.000,0.000,0.000,0.0,1.0
2,Brian Dozier,572821,120,92.0,58.0,38.0,17.0,0.0,14.0,83.0,...,5.88,2.77,1.0,0.333,3.0,0.333,0.667,1.000,0.0,1.0
3,Gerardo Parra,467827,120,74.0,51.0,22.0,9.0,0.0,5.0,39.0,...,5.88,2.77,0.0,0.000,0.0,0.000,0.000,0.000,1.0,1.0
4,Howie Kendrick,435062,120,79.0,67.0,40.0,13.0,0.0,12.0,32.0,...,5.88,2.77,0.0,0.000,0.0,0.000,0.000,0.000,0.0,0.0


In [450]:
player_stats_table.to_csv("player_stats_7_24_19.csv")

In [182]:
hydrate = 'stats(group=[hitting],type=[careerStatSplits], sitCodes=[ven])'
params = {'personId': get_player_id_from_name("Buster Posey"), 'hydrate':hydrate, 'sportId':1}
r = statsapi.get('person',params)
r

ValueError: Request failed. Status Code: 500.

In [107]:
batting_past_N_days(22, get_player_id_from_name("Buster Posey"))

{'avg_p22d': 0.367,
 'hits_p22d': 18.0,
 'obp_p22d': 0.426,
 'ops_p22d': 0.977,
 'slg_p22d': 0.551}

In [108]:
get_h2h_vs_pitcher(get_player_id_from_name("Buster Posey"), get_player_id_from_name("Clayton Kershaw"))

{'avg_h2h': 0.225,
 'hits_h2h': 25.0,
 'obp_h2h': 0.271,
 'ops_h2h': 0.604,
 'slg_h2h': 0.333}

In [45]:
pitching_past_N_days(30, get_player_id_from_name("Wade LeBlanc"))

{'airOuts': 26,
 'atBats': 103,
 'avg': '.252',
 'balks': 0,
 'baseOnBalls': 7,
 'battersFaced': 110,
 'blownSaves': 0,
 'catchersInterference': 0,
 'caughtStealing': 0,
 'completeGames': 0,
 'doubles': 2,
 'earnedRuns': 10,
 'era': '3.38',
 'gamesPitched': 5,
 'gamesPlayed': 5,
 'gamesStarted': 1,
 'groundIntoDoublePlay': 2,
 'groundOuts': 30,
 'groundOutsToAirouts': '0.71',
 'hitBatsmen': 0,
 'hits': 26,
 'hitsPer9Inn': '8.78',
 'holds': 0,
 'homeRuns': 6,
 'homeRunsPer9': '2.06',
 'inheritedRunners': 5,
 'inheritedRunnersScored': 4,
 'inningsPitched': '26.2',
 'intentionalWalks': 1,
 'losses': 1,
 'numberOfPitches': 432,
 'outs': 80,
 'pickoffs': 0,
 'pitchesPerInning': '16.20',
 'runs': 10,
 'runsScoredPer9': '3.44',
 'sacBunts': 0,
 'sacFlies': 0,
 'saveOpportunities': 0,
 'saves': 0,
 'shutouts': 0,
 'stolenBasePercentage': '1.000',
 'stolenBases': 2,
 'strikeOuts': 21,
 'strikePercentage': '.650',
 'strikeoutWalkRatio': '3.00',
 'strikeoutsPer9Inn': '7.09',
 'strikes': 281,
 'tr

In [58]:
check_pitcher_right_handed(get_player_id_from_name("Lance Lynn"))

True

In [69]:
check_batter_right_handed(get_player_id_from_name("Bryce Harper"))

False

In [72]:
check_pitcher_batter_opposite_hand(get_player_id_from_name("Bryce Harper"), get_player_id_from_name("Jacob DeGrom"))

True

In [110]:
statsapi.last_game(teams_dict['sf'])

566518

In [141]:
statsapi.get('schedule', {'sportId': '1', 'gamePk': 566516, 'hydrate':'probablePitcher'})['dates'][0]['games'][0]['teams']['home']['probablePitcher']

{'fullName': 'Samardzija, Jeff', 'id': 502188, 'link': '/api/v1/people/502188'}

In [137]:
statsapi.lookup_player("Buster Posey")

[{'boxscoreName': 'Posey',
  'currentTeam': {'id': 137},
  'firstLastName': 'Buster Posey',
  'firstName': 'Gerald',
  'fullFMLName': 'Gerald Dempsey Posey',
  'fullLFMName': 'Posey, Gerald Dempsey',
  'fullName': 'Buster Posey',
  'id': 457763,
  'initLastName': 'B Posey',
  'lastFirstName': 'Posey, Buster',
  'lastInitName': 'Posey, B',
  'lastName': 'Posey',
  'mlbDebutDate': '2009-09-11',
  'nameFirstLast': 'Buster Posey',
  'primaryNumber': '28',
  'primaryPosition': {'abbreviation': 'C', 'code': '2'},
  'useName': 'Buster'}]

In [144]:
get_opposing_pitcher(457763, 566516)

'Flaherty, Jack'

In [151]:
statsapi.schedule('07/07/2019')

[{'away_id': 110,
  'away_name': 'Baltimore Orioles',
  'away_pitcher_note': 'The Orioles changed their plans on Saturday, opting for Wojciechowski  in this start instead of Gabriel Ynoa. Wojciechowski has made one start since joining Baltimore, but posted a 3.61 ERA over 15 starts in Triple-A with Cleveland earlier this season.',
  'away_probable_pitcher': 'Wojciechowski, Asher',
  'away_score': 1,
  'current_inning': 9,
  'doubleheader': 'N',
  'game_date': '2019-07-07',
  'game_datetime': '2019-07-07T17:07:00Z',
  'game_id': 566911,
  'game_num': 1,
  'game_type': 'R',
  'home_id': 141,
  'home_name': 'Toronto Blue Jays',
  'home_pitcher_note': 'Thornton will take the mound in the team’s final game before the All-Star break. The 25-year-old right-hander is coming off the worst start of his career, in which he allowed seven runs on 11 hits over 2 2/3 innings against the Red Sox on Tuesday. He will be looking for a better outing against Baltimore.',
  'home_probable_pitcher': 'Thornto

In [169]:
statsapi.meta("statTypes")

[{'displayName': 'pecota'},
 {'displayName': 'pecotaRos'},
 {'displayName': 'yearByYear'},
 {'displayName': 'yearByYearAdvanced'},
 {'displayName': 'season'},
 {'displayName': 'seasonAdvanced'},
 {'displayName': 'career'},
 {'displayName': 'careerStatSplits'},
 {'displayName': 'gameLog'},
 {'displayName': 'playLog'},
 {'displayName': 'pitchLog'},
 {'displayName': 'metricLog'},
 {'displayName': 'metricAverages'},
 {'displayName': 'pitchArsenal'},
 {'displayName': 'outsAboveAverage'},
 {'displayName': 'sprayChart'},
 {'displayName': 'vsPlayer'},
 {'displayName': 'vsPlayerTotal'},
 {'displayName': 'vsPlayer5Y'},
 {'displayName': 'vsTeam'},
 {'displayName': 'vsTeam5Y'},
 {'displayName': 'vsTeamTotal'},
 {'displayName': 'lastXGames'},
 {'displayName': 'byDateRange'},
 {'displayName': 'byMonth'},
 {'displayName': 'byDayOfWeek'},
 {'displayName': 'rankings'},
 {'displayName': 'rankingsByYear'},
 {'displayName': 'hotColdZones'},
 {'displayName': 'availableStats'},
 {'displayName': 'opponentsFa